<a href="https://colab.research.google.com/github/Phoenix-7Theta-Fi/AridWigglyTransformations/blob/main/crawl4ai_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crawl4AI: Advanced Web Crawling and Data Extraction

Welcome to this interactive notebook showcasing Crawl4AI, an advanced asynchronous web crawling and data extraction library.

- GitHub Repository: [https://github.com/unclecode/crawl4ai](https://github.com/unclecode/crawl4ai)
- Twitter: [@unclecode](https://twitter.com/unclecode)
- Website: [https://crawl4ai.com](https://crawl4ai.com)

Let's explore the powerful features of Crawl4AI!

## Installation

First, let's install Crawl4AI from GitHub:

In [ ]:
!sudo apt-get update && sudo apt-get install -y libwoff1 libopus0 libwebp6 libwebpdemux2 libenchant1c2a libgudev-1.0-0 libsecret-1-0 libhyphen0 libgdk-pixbuf2.0-0 libegl1 libnotify4 libxslt1.1 libevent-2.1-7 libgles2 libvpx6 libxcomposite1 libatk1.0-0 libatk-bridge2.0-0 libepoxy0 libgtk-3-0 libharfbuzz-icu0

In [ ]:
# !pip install "crawl4ai @ git+https://github.com/unclecode/crawl4ai.git"
# !pip install "crawl4ai @ git+https://github.com/unclecode/crawl4ai.git@staging"
!pip install crawl4ai
!pip install nest-asyncio
!playwright install

Now, let's import the necessary libraries:

In [ ]:
import asyncio
import nest_asyncio
from crawl4ai import AsyncWebCrawler
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy, LLMExtractionStrategy
import json
import time
from pydantic import BaseModel, Field

nest_asyncio.apply()

## Basic Usage

Let's start with a simple crawl example:

In [ ]:
async def simple_crawl():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(url="https://www.nbcnews.com/business")
        print(len(result.markdown))
await simple_crawl()

💡 By default, **Crawl4AI** caches the result of every URL, so the next time you call it, you’ll get an instant result. But if you want to bypass the cache, just set `bypass_cache=True`.

## Advanced Features

### Executing JavaScript and Using CSS Selectors

In [ ]:
async def js_and_css():
    print("\n--- Executing JavaScript and Using CSS Selectors ---")
    # New code to handle the wait_for parameter
    wait_for = """() => {
        return Array.from(document.querySelectorAll('article.tease-card')).length > 10;
    }"""

    # wait_for can be also just a css selector
    # wait_for = "article.tease-card:nth-child(10)"

    async with AsyncWebCrawler(verbose=True) as crawler:
        js_code = [
            "const loadMoreButton = Array.from(document.querySelectorAll('button')).find(button => button.textContent.includes('Load More')); loadMoreButton && loadMoreButton.click();"
        ]
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            js_code=js_code,
            css_selector="article.tease-card",
            # wait_for=wait_for,
            bypass_cache=True,
        )
        print(result.markdown[:500])  # Print first 500 characters

await js_and_css()

### Using a Proxy

Note: You'll need to replace the proxy URL with a working proxy for this example to run successfully.

In [ ]:
async def use_proxy():
    async with AsyncWebCrawler(verbose=True, proxy="http://your-proxy-url:port") as crawler:
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            bypass_cache=True
        )
        print(result.markdown[:500])  # Print first 500 characters

# Uncomment the following line to run the proxy example
# await use_proxy()

### Extracting Structured Data with OpenAI

Note: You'll need to set your OpenAI API key as an environment variable for this example to work.

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

class OpenAIModelFee(BaseModel):
    model_name: str = Field(..., description="Name of the OpenAI model.")
    input_fee: str = Field(..., description="Fee for input token for the OpenAI model.")
    output_fee: str = Field(..., description="Fee for output token for the OpenAI model.")

async def extract_openai_fees():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url='https://openai.com/api/pricing/',
            word_count_threshold=1,
            extraction_strategy=LLMExtractionStrategy(
                provider="openai/gpt-4o", api_token=os.getenv('OPENAI_API_KEY'),
                schema=OpenAIModelFee.schema(),
                extraction_type="schema",
                instruction="""From the crawled content, extract all mentioned model names along with their fees for input and output tokens.
                Do not miss any models in the entire content. One extracted model JSON format should look like this:
                {"model_name": "GPT-4", "input_fee": "US$10.00 / 1M tokens", "output_fee": "US$30.00 / 1M tokens"}."""
            ),
            bypass_cache=True,
        )
        print(len(result.extracted_content))

# Uncomment the following line to run the OpenAI extraction example
await extract_openai_fees()

[LOG] 🌤️  Warming up the AsyncWebCrawler
[LOG] 🌞 AsyncWebCrawler is ready to crawl
[LOG] 🕸️ Crawling https://openai.com/api/pricing/ using AsyncPlaywrightCrawlerStrategy...
[LOG] ✅ Crawled https://openai.com/api/pricing/ successfully!
[LOG] 🚀 Crawling done for https://openai.com/api/pricing/, success: True, time taken: 3.77 seconds
[LOG] 🚀 Content extracted for https://openai.com/api/pricing/, success: True, time taken: 0.21 seconds
[LOG] 🔥 Extracting semantic blocks for https://openai.com/api/pricing/, Strategy: AsyncWebCrawler
[LOG] Call LLM for https://openai.com/api/pricing/ - block index: 0
[LOG] Call LLM for https://openai.com/api/pricing/ - block index: 1
[LOG] Call LLM for https://openai.com/api/pricing/ - block index: 2
[LOG] Call LLM for https://openai.com/api/pricing/ - block index: 3
[LOG] Extracted 4 blocks from URL: https://openai.com/api/pricing/ block index: 3
[LOG] Call LLM for https://openai.com/api/pricing/ - block index: 4
[LOG] Extracted 5 blocks from URL: https://

### Advanced Multi-Page Crawling with JavaScript Execution

## Advanced Multi-Page Crawling with JavaScript Execution

This example demonstrates Crawl4AI's ability to handle complex crawling scenarios, specifically extracting commits from multiple pages of a GitHub repository. The challenge here is that clicking the "Next" button doesn't load a new page, but instead uses asynchronous JavaScript to update the content. This is a common hurdle in modern web crawling.

To overcome this, we use Crawl4AI's custom JavaScript execution to simulate clicking the "Next" button, and implement a custom hook to detect when new data has loaded. Our strategy involves comparing the first commit's text before and after "clicking" Next, waiting until it changes to confirm new data has rendered. This showcases Crawl4AI's flexibility in handling dynamic content and its ability to implement custom logic for even the most challenging crawling tasks.

In [ ]:
import re
from bs4 import BeautifulSoup

async def crawl_dynamic_content_pages_method_3():
    print("\n--- Advanced Multi-Page Crawling with JavaScript Execution using `wait_for` ---")

    async with AsyncWebCrawler(verbose=True) as crawler:
        url = "https://github.com/microsoft/TypeScript/commits/main"
        session_id = "typescript_commits_session"
        all_commits = []

        js_next_page = """
        const commits = document.querySelectorAll('li.Box-sc-g0xbh4-0 h4');
        if (commits.length > 0) {
            window.firstCommit = commits[0].textContent.trim();
        }
        const button = document.querySelector('a[data-testid="pagination-next-button"]');
        if (button) button.click();
        """

        wait_for = """() => {
            const commits = document.querySelectorAll('li.Box-sc-g0xbh4-0 h4');
            if (commits.length === 0) return false;
            const firstCommit = commits[0].textContent.trim();
            return firstCommit !== window.firstCommit;
        }"""

        schema = {
            "name": "Commit Extractor",
            "baseSelector": "li.Box-sc-g0xbh4-0",
            "fields": [
                {
                    "name": "title",
                    "selector": "h4.markdown-title",
                    "type": "text",
                    "transform": "strip",
                },
            ],
        }
        extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

        for page in range(3):  # Crawl 3 pages
            result = await crawler.arun(
                url=url,
                session_id=session_id,
                css_selector="li.Box-sc-g0xbh4-0",
                extraction_strategy=extraction_strategy,
                js_code=js_next_page if page > 0 else None,
                wait_for=wait_for if page > 0 else None,
                js_only=page > 0,
                bypass_cache=True,
                headless=False,
            )

            assert result.success, f"Failed to crawl page {page + 1}"

            commits = json.loads(result.extracted_content)
            all_commits.extend(commits)

            print(f"Page {page + 1}: Found {len(commits)} commits")

        await crawler.crawler_strategy.kill_session(session_id)
        print(f"Successfully crawled {len(all_commits)} commits across 3 pages")

await crawl_dynamic_content_pages_method_3()


--- Advanced Multi-Page Crawling with JavaScript Execution using `wait_for` ---
[LOG] 🌤️  Warming up the AsyncWebCrawler
[LOG] 🌞 AsyncWebCrawler is ready to crawl
[LOG] 🕸️ Crawling https://github.com/microsoft/TypeScript/commits/main using AsyncPlaywrightCrawlerStrategy...
[LOG] ✅ Crawled https://github.com/microsoft/TypeScript/commits/main successfully!
[LOG] 🚀 Crawling done for https://github.com/microsoft/TypeScript/commits/main, success: True, time taken: 7.81 seconds


Exception ignored in: <function AsyncPlaywrightCrawlerStrategy.__del__ at 0x7af0a7ed9750>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/crawl4ai/async_crawler_strategy.py", line 113, in __del__
    asyncio.get_event_loop().run_until_complete(self.close())
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 98, in run_until_complete
    return f.result()
  File "/usr/lib/python3.10/asyncio/futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.10/asyncio/tasks.py", line 234, in __step
    result = coro.throw(exc)
  File "/usr/local/lib/python3.10/dist-packages/crawl4ai/async_crawler_strategy.py", line 108, in close
    await self.playwright.stop()
  File "/usr/local/lib/python3.10/dist-packages/playwright/async_api/_context_manager.py", line 57, in __aexit__
    await self._connection.stop_async()
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_connec

[LOG] 🚀 Content extracted for https://github.com/microsoft/TypeScript/commits/main, success: True, time taken: 0.75 seconds
[LOG] 🔥 Extracting semantic blocks for https://github.com/microsoft/TypeScript/commits/main, Strategy: AsyncWebCrawler
[LOG] 🚀 Extraction done for https://github.com/microsoft/TypeScript/commits/main, time taken: 1.07 seconds.
Page 1: Found 35 commits
[LOG] 🕸️ Crawling https://github.com/microsoft/TypeScript/commits/main using AsyncPlaywrightCrawlerStrategy...
[LOG] ✅ Crawled https://github.com/microsoft/TypeScript/commits/main successfully!
[LOG] 🚀 Crawling done for https://github.com/microsoft/TypeScript/commits/main, success: True, time taken: 2.05 seconds
[LOG] 🚀 Content extracted for https://github.com/microsoft/TypeScript/commits/main, success: True, time taken: 0.59 seconds
[LOG] 🔥 Extracting semantic blocks for https://github.com/microsoft/TypeScript/commits/main, Strategy: AsyncWebCrawler
[LOG] 🚀 Extraction done for https://github.com/microsoft/TypeScript

### Using JsonCssExtractionStrategy for Fast Structured Output

The JsonCssExtractionStrategy is a powerful feature of Crawl4AI that allows for precise, structured data extraction from web pages. Here's how it works:

1. You define a schema that describes the pattern of data you're interested in extracting.
2. The schema includes a base selector that identifies repeating elements on the page.
3. Within the schema, you define fields, each with its own selector and type.
4. These field selectors are applied within the context of each base selector element.
5. The strategy supports nested structures, lists within lists, and various data types.
6. You can even include computed fields for more complex data manipulation.

This approach allows for highly flexible and precise data extraction, transforming semi-structured web content into clean, structured JSON data. It's particularly useful for extracting consistent data patterns from pages like product listings, news articles, or search results.

For more details and advanced usage, check out the full documentation on the Crawl4AI website.

In [ ]:
async def extract_news_teasers():
    schema = {
        "name": "News Teaser Extractor",
        "baseSelector": ".wide-tease-item__wrapper",
        "fields": [
            {
                "name": "category",
                "selector": ".unibrow span[data-testid='unibrow-text']",
                "type": "text",
            },
            {
                "name": "headline",
                "selector": ".wide-tease-item__headline",
                "type": "text",
            },
            {
                "name": "summary",
                "selector": ".wide-tease-item__description",
                "type": "text",
            },
            {
                "name": "time",
                "selector": "[data-testid='wide-tease-date']",
                "type": "text",
            },
            {
                "name": "image",
                "type": "nested",
                "selector": "picture.teasePicture img",
                "fields": [
                    {"name": "src", "type": "attribute", "attribute": "src"},
                    {"name": "alt", "type": "attribute", "attribute": "alt"},
                ],
            },
            {
                "name": "link",
                "selector": "a[href]",
                "type": "attribute",
                "attribute": "href",
            },
        ],
    }

    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            extraction_strategy=extraction_strategy,
            bypass_cache=True,
        )

        assert result.success, "Failed to crawl the page"

        news_teasers = json.loads(result.extracted_content)
        print(f"Successfully extracted {len(news_teasers)} news teasers")
        print(json.dumps(news_teasers[0], indent=2))

await extract_news_teasers()

[LOG] 🌤️  Warming up the AsyncWebCrawler
[LOG] 🌞 AsyncWebCrawler is ready to crawl
[LOG] 🕸️ Crawling https://www.nbcnews.com/business using AsyncPlaywrightCrawlerStrategy...
[LOG] ✅ Crawled https://www.nbcnews.com/business successfully!
[LOG] 🚀 Crawling done for https://www.nbcnews.com/business, success: True, time taken: 6.66 seconds
[LOG] 🚀 Content extracted for https://www.nbcnews.com/business, success: True, time taken: 0.33 seconds
[LOG] 🔥 Extracting semantic blocks for https://www.nbcnews.com/business, Strategy: AsyncWebCrawler
[LOG] 🚀 Extraction done for https://www.nbcnews.com/business, time taken: 0.52 seconds.
Successfully extracted 11 news teasers
{
  "category": "Travel",
  "headline": "Extra legroom, assigned seats, overnight flights: Inside Southwest Airlines' plan for its future",
  "summary": "Southwest said its three-year plan will add $4 billion to earnings before interest and taxes in 2027.",
  "time": "20h ago",
  "image": {
    "src": "https://media-cldnry.s-nbcnew

## Speed Comparison

Let's compare the speed of Crawl4AI with Firecrawl, a paid service. Note that we can't run Firecrawl in this Colab environment, so we'll simulate its performance based on previously recorded data.

💡 **Note on Speed Comparison:**

The speed test conducted here is running on Google Colab, where the internet speed and performance can vary and may not reflect optimal conditions. When we call Firecrawl's API, we're seeing its best performance, while Crawl4AI's performance is limited by Colab's network speed.

For a more accurate comparison, it's recommended to run these tests on your own servers or computers with a stable and fast internet connection. Despite these limitations, Crawl4AI still demonstrates faster performance in this environment.

If you run these tests locally, you may observe an even more significant speed advantage for Crawl4AI compared to other services.

In [ ]:
!pip install firecrawl

In [ ]:
import os
from google.colab import userdata
os.environ['FIRECRAWL_API_KEY'] = userdata.get('FIRECRAWL_API_KEY')
import time
from firecrawl import FirecrawlApp

async def speed_comparison():
    # Simulated Firecrawl performance
    app = FirecrawlApp(api_key=os.environ['FIRECRAWL_API_KEY'])
    start = time.time()
    scrape_status = app.scrape_url(
    'https://www.nbcnews.com/business',
    params={'formats': ['markdown', 'html']}
    )
    end = time.time()
    print("Firecrawl (simulated):")
    print(f"Time taken: {end - start:.2f} seconds")
    print(f"Content length: {len(scrape_status['markdown'])} characters")
    print(f"Images found: {scrape_status['markdown'].count('cldnry.s-nbcnews.com')}")
    print()

    async with AsyncWebCrawler() as crawler:
        # Crawl4AI simple crawl
        start = time.time()
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            word_count_threshold=0,
            bypass_cache=True,
            verbose=False
        )
        end = time.time()
        print("Crawl4AI (simple crawl):")
        print(f"Time taken: {end - start:.2f} seconds")
        print(f"Content length: {len(result.markdown)} characters")
        print(f"Images found: {result.markdown.count('cldnry.s-nbcnews.com')}")
        print()

        # Crawl4AI with JavaScript execution
        start = time.time()
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            js_code=["const loadMoreButton = Array.from(document.querySelectorAll('button')).find(button => button.textContent.includes('Load More')); loadMoreButton && loadMoreButton.click();"],
            word_count_threshold=0,
            bypass_cache=True,
            verbose=False
        )
        end = time.time()
        print("Crawl4AI (with JavaScript execution):")
        print(f"Time taken: {end - start:.2f} seconds")
        print(f"Content length: {len(result.markdown)} characters")
        print(f"Images found: {result.markdown.count('cldnry.s-nbcnews.com')}")

await speed_comparison()

Firecrawl (simulated):
Time taken: 4.38 seconds
Content length: 41967 characters
Images found: 49

Crawl4AI (simple crawl):
Time taken: 4.22 seconds
Content length: 18221 characters
Images found: 49

Crawl4AI (with JavaScript execution):
Time taken: 9.13 seconds
Content length: 34243 characters
Images found: 89


If you run on a local machine with a proper internet speed:
- Simple crawl: Crawl4AI is typically over 3-4 times faster than Firecrawl.
- With JavaScript execution: Even when executing JavaScript to load more content (potentially doubling the number of images found), Crawl4AI is still faster than Firecrawl's simple crawl.

Please note that actual performance may vary depending on network conditions and the specific content being crawled.

## Conclusion

In this notebook, we've explored the powerful features of Crawl4AI, including:

1. Basic crawling
2. JavaScript execution and CSS selector usage
3. Proxy support
4. Structured data extraction with OpenAI
5. Advanced multi-page crawling with JavaScript execution
6. Fast structured output using JsonCssExtractionStrategy
7. Speed comparison with other services

Crawl4AI offers a fast, flexible, and powerful solution for web crawling and data extraction tasks. Its asynchronous architecture and advanced features make it suitable for a wide range of applications, from simple web scraping to complex, multi-page data extraction scenarios.

For more information and advanced usage, please visit the [Crawl4AI documentation](https://crawl4ai.com/mkdocs/).

Happy crawling!